In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

In [7]:
cd Pickled_from_mongo/

/home/jovyan/capstone-52/Pickled_from_mongo


In [8]:
df = pd.read_pickle('../Pickled_from_mongo/combined_eg_gulf.p')

In [13]:
df.shape

(940, 5)

In [21]:
df.head()

,_id,cleaned_geo,cleaned_name,cleaned_text,class
0,5a26dc22204c9e0253e6b196,None,kamal,أزاي أقول لك كنا زمان ... والماضي كان فى الغيب...,EG
1,5a26dc22204c9e0253e6b197,Egypt,agabdelrehim,هي آراء آه بس أزاي أجويرو منتهي يعني أمال لو م...,EG
2,5a26dc22204c9e0253e6b198,None,ElsndubadE,أنت صيني أزاي تقارن شادي بالخطيب ألي هو الوحيد...,EG
3,5a26dc22204c9e0253e6b199,None,h_sawires,أزاي الناس كانت بتغرد في الخمسينات قبل إختراع ...,EG
4,5a26dc22204c9e0253e6b19a,Egypt,Hagerelmor,التوينز اللي معاها كل الحلو والوحش والمصايب وا...,EG


In [1]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
df_test = df['cleaned_text'].sample()

In [18]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['cleaned_text'])
X_train_counts.shape

(940, 4297)

In [17]:
type(X_train_counts)

scipy.sparse.csr.csr_matrix

In [20]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(940, 4297)

In [23]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, df['class'])

#### Conduct search using two different dialects and see if predicted class is accurate

In [30]:
docs_new = ['الثوره المصريه تحولت من ثورة شارع محدش يزعل', 'ذويه ارفضوا لانه عيار جمبازي مافيه شي وبليس مايكسر اماعينه يامال لضعفه قطو بو سبعة ارواح']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r %s' % (doc, df['class'].sample(10)))
    

'الثوره المصريه تحولت من ثورة شارع محدش يزعل' 362      EG
319    GULF
129    GULF
21       EG
406      EG
383    GULF
496      EG
136    GULF
222      EG
9      GULF
Name: class, dtype: object
'ذويه ارفضوا لانه عيار جمبازي مافيه شي وبليس مايكسر اماعينه يامال لضعفه قطو بو سبعة ارواح' 331    GULF
20     GULF
129      EG
315    GULF
195    GULF
116    GULF
19       EG
375      EG
226    GULF
77       EG
Name: class, dtype: object


__MultinomialNB correctly discriminates between EG and GULF dialects__

In [31]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])

In [32]:
text_clf.fit(df['cleaned_text'], df['class']) 

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

#### Evaluating predictive accuracy of the model.

Tagged tweets from both EG and GULF classes that have not been seen by the training data.

In [33]:
search_sentences = [
{"sentence": "الثوره المصريه تحولت من ثورة شارع محدش يزعل", "class" : "EG"},
{"sentence": "نفسي اكون زيك بعرف اطنشك أو اخليك اخر حاجة و بعد كده اضحك عليك بكلمتين و انت تصدق كل مرة عادي", "class" : "EG"},
{"sentence": "بما أن أغلب اللي متابعني مش بقدر اوصلهم أغلب الوقت. . ف كل يوم هعمل تويته آخر اليوم اللي هيعمل لايك", "class" : "EG"},
{"sentence": "مقاومتنا للأشياء طلعت بتقل مع الزمن، مبقيناش نناهد ف حاجة.. و مش عشان أحنا جامدين قوي. هو حيلنا بس", "class" : "EG"},
{"sentence": "عارف ايه احلى حاجة حاصلة ليا انى منك وانت برضه بتجرى فيا انت اخر كل يوم باخدك ف حضنى وانت اول", "class" : "EG"},
{"sentence": "القاضى اللى حكم على المعتقلين بالاعدام هو هو نفس القاضى اللى هيراقب الانتخابات", "class" : "EG"},
{"sentence": "وفجأة تيجي سيرة حاجة في وسط الكلام تقلب عليك القديم والجديد وترسم في دماغك علامات استفهام مالهاش", "class" : "EG"},
{"sentence": "السنة اللي فاتت الاعلام الانجليزي قال المفروض بيب يعرف انه في البريمييرليج لازم يتأقلم و يلعب كورتنا", "class" : "EG"},
{"sentence": "حرب و قتال و ناس تموت و هذا الدلخ يقول سعيد و مثل أجواء كرة القدم ", "class" : "GULF"},
{"sentence": "من غباء الهلالي الدلخ اللي يفتخر بفوز فريقه من قيادة رئيس الحكام كلاتنبيرغ له سنه ماسنع الحكام السعوديين", "class" : "GULF"},
{"sentence": "شفتوا هوشة شيعان وغالي لو هي بين الهلاليين كان شفتوا هاشتاق كبر راسهم المنسم وكان جاك هذا الدلخ ", "class" : "GULF"},
{"sentence": "ذويه ارفضوا لانه عيار جمبازي مافيه شي وبليس مايكسر اماعينه يامال لضعفه قطو بو سبعة ارواح ", "class" : "GULF"},
{"sentence": "صج ياجماعه في سوال محيرني ليش المتان مافيهم النفسيه عكس الضعاف تقول خاشوقه ومنفس", "class" : "GULF"},
{"sentence": "لم نعاند التاريخ مسيو خاشوقه بل الواقع والعقلانية ابعدنا من التدمير والانفلات", "class" : "GULF"},
{"sentence": "أي والله وعندي عنه ابو خاشوقة أسرار لا تشرف قد أقولها اذا لم يلجم لسانه عن سب وطني", "class" : "GULF"},
{"sentence": "قبل ماتتكلمين يالطيبه افهمي السالفه ومنب ملزومه بسنابي اني اشرح كل شيء صارت بالتفصيل بس لانك قلق خل", "class" : "GULF"},
]

In [34]:
search_sentences_df = pd.DataFrame(search_sentences)

In [35]:
search_sentences_df.sample(4)

,class,sentence
2,EG,بما أن أغلب اللي متابعني مش بقدر اوصلهم أغلب ا...
1,EG,نفسي اكون زيك بعرف اطنشك أو اخليك اخر حاجة و ب...
0,EG,الثوره المصريه تحولت من ثورة شارع محدش يزعل
6,EG,وفجأة تيجي سيرة حاجة في وسط الكلام تقلب عليك ا...


In [37]:
import numpy as np

docs_test = search_sentences_df['sentence']
predicted = text_clf.predict(docs_test)
np.mean(predicted == search_sentences_df['class'])  

0.875

__Comparing to Support Vector Machine (SVM)__

In [41]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42)),
                    ])
text_clf.fit(df['cleaned_text'], df['class'])  

predicted = text_clf.predict(docs_test)
np.mean(predicted == search_sentences_df['class']) 

0.9375

In [50]:
from sklearn import metrics
print(metrics.classification_report(search_sentences_df['class'], predicted, 
                                    target_names=['EG',"GULF"]))

             precision    recall  f1-score   support

         EG       1.00      0.88      0.93         8
       GULF       0.89      1.00      0.94         8

avg / total       0.94      0.94      0.94        16



In [51]:
metrics.confusion_matrix(search_sentences_df['class'], predicted)

array([[7, 1],
       [0, 8]])

__Parameter tuning using grid search__

In [52]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
             }

In [53]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [55]:
gs_clf = gs_clf.fit(df['cleaned_text'], df['class'])

In [57]:
search_sentences_df['class'][gs_clf.predict(['الثوره المصريه تحولت من ثورة شارع محدش يزعل'])]

GULF    NaN
Name: class, dtype: object

In [58]:
gs_clf.best_score_  

0.89255319148936174

In [59]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 2)


In [62]:
cv_results_df = pd.DataFrame(gs_clf.cv_results_)

In [64]:
high_mean_test_mask = cv_results_df['mean_test_score'] >= 0.87

In [69]:
for i, word in iterrows.high_mean_test_mask:
    print(i, word)

NameError: name 'iterrows' is not defined